# Performance Benchmarking

**Yuning Zhang, TU Delft, 2021.10.27**

In [2]:
include("SpinEnsemble.jl")

FID_plot_options

In [3]:
D=rand_bath_dipolar_coefs(500,3);

In [ ]:
ensemble_FID(t::AbstractVector{<:Real},D::Vector{Float64})=map(x->ensemble_FID(x,D),t);

In [ ]:
@time ensemble_FID

In [ ]:
ensemble_FID(t::AbstractVector{<:Real},D::Vector{Float64})=mapreduce(cos.,*,D*t)/2;